# Model Selection

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
import mlflow
from dotenv import load_dotenv

load_dotenv()

ROOT_DIR = os.path.join(os.getcwd(), '..')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
mlflow.set_experiment('Imbalanced Credit Card Fraud Detection')
mlflow.sklearn.autolog()

## Choosing the metrics

In [6]:
def log_model_results(y_true, y_hat):
    """Calculate, print, and log metrics and artifacts to MLflow."""

    recall = recall_score(y_true, y_hat)
    f1 = f1_score(y_true, y_hat)
    precision = precision_score(y_true, y_hat, zero_division=1)
    accuracy = accuracy_score(y_true, y_hat)

    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    metrics_dict = {
        "test_recall": recall,
        "test_f1_score": f1,
        "test_precision": precision,
        "test_accuracy": accuracy
    }
    mlflow.log_metrics(metrics_dict)

    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_true, y_hat, labels=[0, 1])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Fraud (P)', 'Fraud (P)'],
                yticklabels=['No Fraud (T)', 'Fraud (T)'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.plot()

    confusion_matrix_path = "confusion_matrix.png"
    plt.savefig(confusion_matrix_path)
    plt.close()

    mlflow.log_artifact(confusion_matrix_path, "plots")

## Establishing a baseline

In [7]:
df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [8]:
with mlflow.start_run(run_name='Baseline Model'):

    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat = baseline.predict(df_test.drop(columns=['Class']))

    log_model_results(df_test['Class'], y_hat)

2025/10/13 23:32:13 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\git_reps\imbalanced_credit_fraud_detection\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Recall: 0.0000
F1 Score: 0.0000
Precision: 1.0000
Accuracy: 0.9983
🏃 View run Baseline Model at: http://localhost:5000/#/experiments/713154386041343957/runs/42c9e90c82434f0dbaf1c0a06644b092
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957
